In [1]:
# list of imports
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import copy
from sklearn.feature_selection import VarianceThreshold
import tensorflow as tf
from tensorflow.keras import layers, models
import plotly.express as px
import seaborn as sns


Importing and Preparing the Dataset
Download the `IDS_Dataset.zip` file, which contains all the necessary .csv files for training and testing:

In [2]:
!gdown 15bfx9AMWCCXLWDcX1LBraiwwP_YiNIv6

'gdown' is not recognized as an internal or external command,
operable program or batch file.


https://drive.usercontent.google.com/download?id=15bfx9AMWCCXLWDcX1LBraiwwP_YiNIv6&authuser=0


In [3]:
import zipfile

# Unzip the archive
local_zip = './IDS_Dataset.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall()

zip_ref.close()

In [4]:
import os


archives = os.listdir('IDS_Dataset')


df_IDS_Files = pd.DataFrame({'Archive': archives})
df_IDS_Files

,Archive
0,Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv
1,Friday-WorkingHours-Afternoon-PortScan.pcap_IS...
2,Friday-WorkingHours-Morning.pcap_ISCX.csv
3,Monday-WorkingHours.pcap_ISCX.csv
4,Thursday-WorkingHours-Afternoon-Infilteration....
5,Thursday-WorkingHours-Morning-WebAttacks.pcap_...
6,Tuesday-WorkingHours.pcap_ISCX.csv
7,Wednesday-workingHours.pcap_ISCX.csv


Create a Pandas DataFrame that contains all the available data from the previous dataset:


In [5]:
datasets = 'IDS_Dataset'

df_IDS_combine = []
for archive in df_IDS_Files['Archive']:
    new_archive = os.path.join(datasets, archive)
    df_temp = pd.read_csv(new_archive)
    df_IDS_combine.append(df_temp)

df_IDS_all = pd.concat(df_IDS_combine, ignore_index=True)

del df_IDS_combine
del df_temp

Display the resulting combined DataFrame:

In [6]:
from sklearn.utils import shuffle
df_IDS_all = shuffle(df_IDS_all, random_state=3)

df_IDS_all.head(10)

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
1090143,53,175,2,2,62,122,31,31,31.000000,0.000000,...,40,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
67513,80,1883218,3,4,26,11607,20,0,8.666667,10.263203,...,20,0.0,0.0,0,0,0.0,0.0,0,0,DDoS
2113229,53,4494027,2,2,115,281,62,53,57.500000,6.363961,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2599722,59470,11944,2,1,280,6,274,6,140.000000,189.504617,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1822668,53,206435,1,1,40,166,40,40,40.000000,0.000000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
179581,80,371170,3,5,26,11607,20,0,8.666667,10.263203,...,20,0.0,0.0,0,0,0.0,0.0,0,0,DDoS
661395,443,5097077,8,6,372,3856,191,0,46.500000,71.876879,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
533657,35198,208,2,1,31,6,31,0,15.500000,21.920310,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2502610,443,96923,8,5,547,4648,194,0,68.375000,80.092692,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
244343,443,65215337,24,29,3396,5871,1220,0,141.500000,271.353612,...,20,6654764.0,0.0,6654764,6654764,58500000.0,0.0,58500000,58500000,BENIGN


Create the dataset `df_ids`, which will be used for all subsequent cleaning and curating tasks. This dataset will contain all the records from the merged dataset, ensuring a balanced number of positive and negative (0/1) labels once the complete dataset is partitioned into various sub-datasets for model training.


In [7]:
print('Shape of imported Pandas list =',df_IDS_all.shape)

df_ids = df_IDS_all.drop_duplicates().dropna()

print('Shape of processed Pandas list =',df_ids.shape)

Shape of imported Pandas list = (2830743, 79)
Shape of processed Pandas list = (2522009, 79)


In [8]:
del df_IDS_all

Display the description of each column in the dataset using `.describe()`:

In [9]:
df_ids.describe()

C:\Users\vasum\AppData\Roaming\Python\Python312\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
C:\Users\vasum\AppData\Roaming\Python\Python312\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
count,2.522009e+06,2.522009e+06,2.522009e+06,2.522009e+06,2.522009e+06,2.522009e+06,2.522009e+06,2.522009e+06,2.522009e+06,2.522009e+06,...,2.522009e+06,2.522009e+06,2.522009e+06,2.522009e+06,2.522009e+06,2.522009e+06,2.522009e+06,2.522009e+06,2.522009e+06,2.522009e+06
mean,8.701432e+03,1.658364e+07,1.027750e+01,1.156751e+01,6.116607e+02,1.813569e+04,2.311241e+02,1.919733e+01,6.347899e+01,7.728840e+01,...,6.006745e+00,-3.080742e+03,9.153450e+04,4.616959e+04,1.719345e+05,6.543215e+04,9.332884e+06,5.655225e+05,9.759082e+06,8.888401e+06
std,1.902225e+04,3.522618e+07,7.942294e+02,1.056668e+03,1.058573e+04,2.397602e+06,7.562104e+02,6.079830e+01,1.955137e+02,2.968147e+02,...,6.742531e+02,1.149482e+06,6.864883e+05,4.164856e+05,1.085317e+06,6.110134e+05,2.484306e+07,4.873014e+06,2.561220e+07,2.457630e+07
min,0.000000e+00,-1.300000e+01,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,-5.368707e+08,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,5.300000e+01,2.080000e+02,2.000000e+00,1.000000e+00,1.200000e+01,6.000000e+00,6.000000e+00,0.000000e+00,6.000000e+00,0.000000e+00,...,0.000000e+00,2.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,8.000000e+01,5.058700e+04,2.000000e+00,2.000000e+00,6.600000e+01,1.560000e+02,4.000000e+01,2.000000e+00,3.613084e+01,0.000000e+00,...,1.000000e+00,2.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,4.430000e+02,5.330376e+06,6.000000e+00,5.000000e+00,3.320000e+02,9.910000e+02,2.020000e+02,3.700000e+01,5.200000e+01,7.417179e+01,...,3.000000e+00,3.200000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,6.553500e+04,1.200000e+08,2.197590e+05,2.919220e+05,1.290000e+07,6.554530e+08,2.482000e+04,2.325000e+03,5.940857e+03,7.125597e+03,...,2.135570e+05,1.380000e+02,1.100000e+08,7.420000e+07,1.100000e+08,1.100000e+08,1.200000e+08,7.690000e+07,1.200000e+08,1.200000e+08



---

# Data Cleaning and Curating

---

---

First, we display the names of all columns to determine how to improve their names:



In [10]:
df_ids.columns

Index([' Destination Port', ' Flow Duration', ' Total Fwd Packets',
       ' Total Backward Packets', 'Total Length of Fwd Packets',
       ' Total Length of Bwd Packets', ' Fwd Packet Length Max',
       ' Fwd Packet Length Min', ' Fwd Packet Length Mean',
       ' Fwd Packet Length Std', 'Bwd Packet Length Max',
       ' Bwd Packet Length Min', ' Bwd Packet Length Mean',
       ' Bwd Packet Length Std', 'Flow Bytes/s', ' Flow Packets/s',
       ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min',
       'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max',
       ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean', ' Bwd IAT Std',
       ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags', ' Bwd PSH Flags',
       ' Fwd URG Flags', ' Bwd URG Flags', ' Fwd Header Length',
       ' Bwd Header Length', 'Fwd Packets/s', ' Bwd Packets/s',
       ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean',
       ' Packet Length Std', ' Packet Length Variance', '

We can see that many of them start with an empty field. We change the names of all columns that contain an empty character at the beginning and remove that space.

To make the names more manageable in subsequent code, we replace all spaces (" ") in the column names with underscores ("_").

In [11]:
for column in df_ids.columns:
    new_column_name = column.strip().replace(" ", "_") 
    df_ids.rename(columns={column: new_column_name}, inplace=True)  

print(df_ids.columns)

Index(['Destination_Port', 'Flow_Duration', 'Total_Fwd_Packets',
       'Total_Backward_Packets', 'Total_Length_of_Fwd_Packets',
       'Total_Length_of_Bwd_Packets', 'Fwd_Packet_Length_Max',
       'Fwd_Packet_Length_Min', 'Fwd_Packet_Length_Mean',
       'Fwd_Packet_Length_Std', 'Bwd_Packet_Length_Max',
       'Bwd_Packet_Length_Min', 'Bwd_Packet_Length_Mean',
       'Bwd_Packet_Length_Std', 'Flow_Bytes/s', 'Flow_Packets/s',
       'Flow_IAT_Mean', 'Flow_IAT_Std', 'Flow_IAT_Max', 'Flow_IAT_Min',
       'Fwd_IAT_Total', 'Fwd_IAT_Mean', 'Fwd_IAT_Std', 'Fwd_IAT_Max',
       'Fwd_IAT_Min', 'Bwd_IAT_Total', 'Bwd_IAT_Mean', 'Bwd_IAT_Std',
       'Bwd_IAT_Max', 'Bwd_IAT_Min', 'Fwd_PSH_Flags', 'Bwd_PSH_Flags',
       'Fwd_URG_Flags', 'Bwd_URG_Flags', 'Fwd_Header_Length',
       'Bwd_Header_Length', 'Fwd_Packets/s', 'Bwd_Packets/s',
       'Min_Packet_Length', 'Max_Packet_Length', 'Packet_Length_Mean',
       'Packet_Length_Std', 'Packet_Length_Variance', 'FIN_Flag_Count',
       'SYN_Flag_Co

As we can see, the formats of the names are more uniform and follow a logical pattern.

Before creating a correlation matrix, we remove fields with n/a values and assign values of 0 or 1 to the target category 'Label':


In [12]:
df_ids.dropna(inplace=True)

df_ids['Label'] = df_ids['Label'].replace({'BENIGN': 0})

df_ids['Label'] = df_ids['Label'].apply(lambda x: 1 if x != 0 else x)


We create an auxiliary dataset (*subset*) that contains only the first 500,000 entries to perform exploratory analysis more quickly:



In [13]:
df_ids_subset = df_ids.head(500000)

We draw the correlation matrix using the `plotly` library, which allows us to see explicit values by moving the cursor over the cell and zoom in to observe the values more closely if necessary.


In [14]:
df_ids_corr = df_ids_subset.corr()
fig = px.imshow(df_ids_corr, color_continuous_scale='Viridis', labels=dict(color='Correlation'))
fig.update_layout(title='Matrix', width=1000, height=900)
fig.show()

We remove columns from the correlation matrix that contain NaN values (shown in gray) and regenerate the matrix:


In [15]:
df_ids_filtered = df_ids_subset.copy()
df_ids_filtered.drop(columns=['Fwd_URG_Flags','CWE_Flag_Count','Bwd_Avg_Bytes/Bulk','Fwd_Avg_Bulk_Rate','Bwd_Avg_Packets/Bulk','Bwd_URG_Flags',
                     'Bwd_PSH_Flags','Bwd_Avg_Bulk_Rate','Fwd_Avg_Bytes/Bulk','Fwd_Avg_Packets/Bulk'], inplace=True)
df_ids_filtered.shape

(500000, 69)

We provide the correlation matrix with `plotly` after removing the previously mentioned features:


In [16]:
df_ids_filtered_corr = df_ids_filtered.corr()
fig = px.imshow(df_ids_filtered_corr, color_continuous_scale='Viridis', labels=dict(color='Correlation'))
fig.update_layout(title='Matrix', width=1000, height=900)
fig.show()

In [17]:
columns_eliminate = [column for column in df_ids_filtered.columns if 'Min' in column or 'Max' in column]
df_ids_filtered.drop(columns=columns_eliminate, inplace=True)

print('Características eliminates:')
print(columns_eliminate)
print(len(columns_eliminate))

Características eliminates:
['Fwd_Packet_Length_Max', 'Fwd_Packet_Length_Min', 'Bwd_Packet_Length_Max', 'Bwd_Packet_Length_Min', 'Flow_IAT_Max', 'Flow_IAT_Min', 'Fwd_IAT_Max', 'Fwd_IAT_Min', 'Bwd_IAT_Max', 'Bwd_IAT_Min', 'Min_Packet_Length', 'Max_Packet_Length', 'Active_Max', 'Active_Min', 'Idle_Max', 'Idle_Min']
16


In [18]:
df_ids_filtered_corr = df_ids_filtered.corr()
fig = px.imshow(df_ids_filtered_corr, color_continuous_scale='Viridis', labels=dict(color='Correlation'))
fig.update_layout(title='Matrix', width=1000, height=900)
fig.show()

print('Number of characteristics:')
print(df_ids_filtered.shape[1])


Number of characteristics:
53


We now perform a screening based solely on the correlation value of the dataset's features with the *target* 'Label'.
All features with a correlation of less than 0.2 with 'Label' (our probability of attack) will be discarded.


In [19]:
correlaciones_label = df_ids_filtered_corr.loc['Label']

charcteristics_eliminate = correlaciones_label[abs(correlaciones_label) < 0.2].index.tolist()

df_ids_filtered = df_ids_filtered.drop(columns=charcteristics_eliminate)

print("Characteristics eliminates:")
print(charcteristics_eliminate)
print("Number of characteristics eliminates:")
print(len(charcteristics_eliminate))

Characteristics eliminates:
['Destination_Port', 'Total_Fwd_Packets', 'Total_Backward_Packets', 'Total_Length_of_Fwd_Packets', 'Total_Length_of_Bwd_Packets', 'Fwd_Packet_Length_Mean', 'Fwd_Packet_Length_Std', 'Flow_Bytes/s', 'Flow_Packets/s', 'Fwd_IAT_Mean', 'Bwd_IAT_Total', 'Bwd_IAT_Mean', 'Bwd_IAT_Std', 'Fwd_PSH_Flags', 'Fwd_Header_Length', 'Bwd_Header_Length', 'Fwd_Packets/s', 'Bwd_Packets/s', 'SYN_Flag_Count', 'RST_Flag_Count', 'PSH_Flag_Count', 'ACK_Flag_Count', 'URG_Flag_Count', 'ECE_Flag_Count', 'Down/Up_Ratio', 'Avg_Fwd_Segment_Size', 'Fwd_Header_Length.1', 'Subflow_Fwd_Packets', 'Subflow_Fwd_Bytes', 'Subflow_Bwd_Packets', 'Subflow_Bwd_Bytes', 'Init_Win_bytes_forward', 'Init_Win_bytes_backward', 'act_data_pkt_fwd', 'min_seg_size_forward', 'Active_Mean', 'Active_Std', 'Idle_Std']
Number of characteristics eliminates:
38


In [20]:
df_ids_filtered_corr = df_ids_filtered.corr()
fig = px.imshow(df_ids_filtered_corr, color_continuous_scale='Viridis', labels=dict(color='Correlation'))
fig.update_layout(title='Matrix', width=800, height=800)
fig.show()

print('Number of characteristics:')
print(df_ids_filtered.shape[1])

Number of characteristics:
15


In [21]:
characteristics_of_eliminator_2 = ['Packet_Length_Variance','Average_Packet_Size','Flow_IAT_Mean']
df_ids_filtered = df_ids_filtered.drop(columns=characteristics_of_eliminator_2)

In [22]:
df_ids_var = df_ids_filtered.drop(columns='Label')
variances = df_ids_var.var()

print(variances)

Flow_Duration             1.192635e+15
Bwd_Packet_Length_Mean    3.867578e+05
Bwd_Packet_Length_Std     7.486873e+05
Flow_IAT_Std              6.895042e+13
Fwd_IAT_Total             1.187734e+15
Fwd_IAT_Std               9.839948e+13
Packet_Length_Mean        9.820991e+04
Packet_Length_Std         4.227918e+05
FIN_Flag_Count            2.987957e-02
Avg_Bwd_Segment_Size      3.867578e+05
Idle_Mean                 5.912673e+14
dtype: float64


In [23]:
characteristics_eliminate_3 = ['FIN_Flag_Count']
df_ids_filtered = df_ids_filtered.drop(columns=characteristics_eliminate_3 )

The resulting correlation matrix is as follows:


In [24]:
df_ids_filtered_corr = df_ids_filtered.corr()
fig = px.imshow(df_ids_filtered_corr, color_continuous_scale='Viridis', labels=dict(color='Correlation'))
fig.update_layout(title='Matrix', width=800, height=800)
fig.show()

print('Number of caracteristics:')
print(df_ids_filtered.shape[1])

Number of caracteristics:
11


We now list the surviving features from the screening to select the columns we need from the general dataframe:


In [25]:
columns_df_ids_filtered = df_ids_filtered.columns

print("Columns of DataFrame:")
for columna in columns_df_ids_filtered:
    print(columna)

Columns of DataFrame:
Flow_Duration
Bwd_Packet_Length_Mean
Bwd_Packet_Length_Std
Flow_IAT_Std
Fwd_IAT_Total
Fwd_IAT_Std
Packet_Length_Mean
Packet_Length_Std
Avg_Bwd_Segment_Size
Idle_Mean
Label


From the complete dataframe `df_ids`, we keep only the columns resulting from the feature study and display the first ten records

In [26]:
df_ids = df_ids[columns_df_ids_filtered]
df_ids.head(10)

,Flow_Duration,Bwd_Packet_Length_Mean,Bwd_Packet_Length_Std,Flow_IAT_Std,Fwd_IAT_Total,Fwd_IAT_Std,Packet_Length_Mean,Packet_Length_Std,Avg_Bwd_Segment_Size,Idle_Mean,Label
1090143,175,61.000000,0.000000,6.313742e+01,48,0.000000e+00,43.000000,16.431677,61.000000,0.0,0
67513,1883218,2901.750000,5795.500690,7.683199e+05,711,3.981011e+02,1454.125000,4097.537534,2901.750000,0.0,1
2113229,4494027,140.500000,44.547727,2.469370e+06,4456167,0.000000e+00,91.600000,50.022995,140.500000,0.0,0
2599722,11944,6.000000,0.000000,8.358002e+03,11944,0.000000e+00,140.000000,154.729872,6.000000,0.0,0
1822668,206435,166.000000,0.000000,0.000000e+00,0,0.000000e+00,82.000000,72.746134,166.000000,0.0,0
179581,371170,2321.400000,3327.769794,1.399875e+05,473,2.298097e+02,1292.555556,2650.585648,2321.400000,0.0,1
661395,5097077,642.666667,680.738031,1.345043e+06,5097077,1.825771e+06,281.866667,511.082446,642.666667,0.0,0
533657,208,6.000000,0.000000,4.949747e+01,208,0.000000e+00,17.000000,16.350331,6.000000,0.0,0
2502610,96923,929.600000,852.604715,1.443092e+04,96923,2.462930e+04,371.071429,643.494602,929.600000,0.0,0
244343,65215337,202.448276,406.153612,8.105628e+06,65200000,1.220000e+07,171.611111,347.259971,202.448276,58500000.0,0


 Outlier Analysis

In [27]:
Q1 = df_ids.quantile(0.01)
Q3 = df_ids.quantile(0.99)
IQR = Q3 - Q1

We define limits to identify outliers using the IQR criterion.




In [28]:
lower_bound = Q1 - 1.0 * IQR
upper_bound = Q3 + 1.0 * IQR

In [29]:
outliers = ((df_ids < lower_bound) | (df_ids > upper_bound)).any(axis=1)
count_outliers = outliers.sum()
print(f"Total outliers (True): {count_outliers}")

Total outliers (True): 374


We create a dataset that contains only the outliers:


In [30]:
files_with_outliers = df_ids[outliers]
files_with_outliers

,Flow_Duration,Bwd_Packet_Length_Mean,Bwd_Packet_Length_Std,Flow_IAT_Std,Fwd_IAT_Total,Fwd_IAT_Std,Packet_Length_Mean,Packet_Length_Std,Avg_Bwd_Segment_Size,Idle_Mean,Label
2793355,107703472,6.0,0.0,7.620000e+07,108000000,0.000000e+00,121.25,238.516771,6.0,108000000.0,1
1779574,119996808,185.0,0.0,8.470000e+07,120000000,0.000000e+00,80.00,70.000000,185.0,120000000.0,0
2794201,107647497,6.0,0.0,7.610000e+07,108000000,0.000000e+00,98.25,192.520778,6.0,108000000.0,1
2795001,107654927,6.0,0.0,7.610000e+07,108000000,0.000000e+00,114.25,224.517817,6.0,108000000.0,1
2795368,107785036,6.0,0.0,7.620000e+07,108000000,0.000000e+00,154.25,304.513136,6.0,108000000.0,1
...,...,...,...,...,...,...,...,...,...,...,...
2615644,119846787,124.0,0.0,8.460000e+07,120000000,0.000000e+00,65.25,39.347808,124.0,120000000.0,0
2794301,107654662,6.0,0.0,7.610000e+07,108000000,0.000000e+00,116.25,228.517505,6.0,108000000.0,1
887220,115491064,0.0,0.0,8.166440e+07,115491064,8.166440e+07,0.00,0.000000,0.0,0.0,0
2792798,107488462,6.0,0.0,7.600000e+07,107000000,0.000000e+00,134.00,264.015151,6.0,107000000.0,1



---

# Data Export

---


We divide the dataset `df_ids` into four datasets of similar size:



In [31]:
total_files = len(df_ids)

div_size = total_files 

datasets_divisions = np.array_split(df_ids, 4)

df_ids_0, df_ids_1, df_ids_2, df_ids_3 = datasets_divisions

C:\Users\vasum\AppData\Roaming\Python\Python312\site-packages\numpy\core\fromnumeric.py:59: FutureWarning:

'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.



We print the number of rows and columns of the resulting dataframes:

In [32]:
print("Shape of df_ids_0", df_ids_0.shape)
print("Shape of df_ids_1", df_ids_1.shape)
print("Shape of df_ids_2", df_ids_2.shape)
print("Shape of df_ids_3", df_ids_3.shape)

Shape of df_ids_0 (630503, 11)
Shape of df_ids_1 (630502, 11)
Shape of df_ids_2 (630502, 11)
Shape of df_ids_3 (630502, 11)


Next, we perform a check of the attack count for each of the resulting dataframes:


In [33]:
def count_label_1(df):
    return (df['Label'] == 1).sum()

print("Number of values having 'Label' equal to 1 in df_ids_0:", count_label_1(df_ids_0))
print("Number of values having 'Label' equal to 1 in df_ids_1:", count_label_1(df_ids_1))
print("Number of values having 'Label' equal to 1 in df_ids_2:", count_label_1(df_ids_2))
print("Number of values having 'Label' equal to 1 in df_ids_3:", count_label_1(df_ids_3))


Number of values having 'Label' equal to 1 in df_ids_0: 112214
Number of values having 'Label' equal to 1 in df_ids_1: 107096
Number of values having 'Label' equal to 1 in df_ids_2: 104247
Number of values having 'Label' equal to 1 in df_ids_3: 102318


We export the files to `.csv` format:

*Note: Each file specifies its number (from 0 to 3) and whether the dataframe is balanced or not (NB: Not balanced, B: Balanced).*


In [34]:
df_ids_0.to_csv('df_ids_0_NB_v1.csv', index=False)
df_ids_1.to_csv('df_ids_1_NB_v1.csv', index=False)
df_ids_2.to_csv('df_ids_2_NB_v1.csv', index=False)
df_ids_3.to_csv('df_ids_3_NB_v1.csv', index=False)

In [35]:
def balance_dataframe(df):
    count_class_0 = (df['Label'] == 0).sum()
    count_class_1 = (df['Label'] == 1).sum()

    min_count_class = min(count_class_0, count_class_1)

    df_class_0 = df[df['Label'] == 0].sample(n=min_count_class, random_state=3)
    df_class_1 = df[df['Label'] == 1].sample(n=min_count_class, random_state=3)

    df_balance = pd.concat([df_class_0, df_class_1])

    return df_balance

df_ids_0_balance = balance_dataframe(df_ids_0)
df_ids_1_balance = balance_dataframe(df_ids_1)
df_ids_2_balance = balance_dataframe(df_ids_2)
df_ids_3_balance = balance_dataframe(df_ids_3)

print("Information of df_ids_0_balance:")
print(df_ids_0_balance['Label'].value_counts())
print()

print("Information of df_ids_1_balance:")
print(df_ids_1_balance['Label'].value_counts())
print()

print("Information of df_ids_2_balance:")
print(df_ids_2_balance['Label'].value_counts())
print()

print("Information of df_ids_3_balance:")
print(df_ids_3_balance['Label'].value_counts())

Information of df_ids_0_balance:
Label
0    112214
1    112214
Name: count, dtype: int64

Information of df_ids_1_balance:
Label
0    107096
1    107096
Name: count, dtype: int64

Information of df_ids_2_balance:
Label
0    104247
1    104247
Name: count, dtype: int64

Information of df_ids_3_balance:
Label
0    102318
1    102318
Name: count, dtype: int64


We export these four balanced dataframes to `.csv` files:


In [36]:
df_ids_0_balance.to_csv('df_ids_0_B_v1.csv', index=False)
df_ids_1_balance.to_csv('df_ids_1_B_v1.csv', index=False)
df_ids_2_balance.to_csv('df_ids_2_B_v1.csv', index=False)
df_ids_3_balance.to_csv('df_ids_3_B_v1.csv', index=False)

We provide, as a code comment, the `!gdown` statement to download a .zip file containing all the .csv files resulting from the data processing:


We remove data from the previous code (except for the output) to free up RAM:

In [37]:
del df_ids